In [ ]:
# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [4]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize as imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [5]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [6]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 52

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [7]:
image_idx = [0,2,4,6,7,8,10,12,15,16,18,20,22,23,24,26,28,29]
idx_len = len(image_idx)
print(idx_len)

image_height = 96
image_width = 96


18


In [8]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = image_idx #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,18,image_height,image_width,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    # Reshaping images and trimming it first from corners if shape has 160 width
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(image_height,image_width)).astype(np.float32)
                    else:
                        image = imresize(image,(image_height,image_width)).astype(np.float32)
                        
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    # Doing max normalization rather than dividing just by 255.
                    batch_data[folder,idx,:,:,0]  = (image[:,:,0]) / (image[:,:,0]).max()
                    batch_data[folder,idx,:,:,1]  = (image[:,:,1]) / (image[:,:,1]).max()
                    batch_data[folder,idx,:,:,2]  = (image[:,:,2]) / (image[:,:,2]).max()
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        rem_sequemce_len = 0
        if((len(t)%batch_size)!=0):
            rem_sequemce_len = len(t)%batch_size
        
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((rem_sequemce_len,18,image_height,image_width,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((rem_sequemce_len,5)) # batch_labels is the one hot representation of the output

            for folder in range(rem_sequemce_len): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                   
                    if image.shape[1] == 160:
                        image = imresize(image[:,20:140,:],(image_height,image_width)).astype(np.float32)
                    else:
                        image = imresize(image,(image_height,image_width)).astype(np.float32)
                    #image = imresize(image,(120,120,3))
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #image = image_augmentation(image)
                    
                    batch_data[folder,idx,:,:,0]  = (image[:,:,0]) / (image[:,:,0]).max()
                    batch_data[folder,idx,:,:,1]  = (image[:,:,1]) / (image[:,:,1]).max()
                    batch_data[folder,idx,:,:,2]  = (image[:,:,2]) / (image[:,:,2]).max()
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [9]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [10]:
from tensorflow.keras.applications import mobilenet

In [11]:
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, TimeDistributed, LSTM, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

#write your model here

mobileNet = mobilenet.MobileNet(weights='imagenet', include_top=False)

Input_shape = (idx_len, image_height, image_width, 3)

model = Sequential()
model.add(TimeDistributed(mobileNet,input_shape=Input_shape))


for layer in model.layers:
    layer.trainable = True


model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))

model.add(LSTM(64))
model.add(Dropout(0.40))

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.40))

model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [12]:
optimiser = tf.optimizers.Adam(learning_rate=1e-2)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 18, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 18, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 18, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 18, 1024)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                278784    
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                4

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [13]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

model_name = 'model_13' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor = 'val_loss',
                                    mode = 'min',
                                    save_best_only = True,
                                    verbose = 1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 3, verbose = 1)
        
#early_stop = EarlyStopping(monitor = "loss", patience=9, restore_best_weights =True, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [15]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [16]:
hist1 = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 52
Epoch 1/50
13/13 [==============================] - ETA: 0s - loss: 1.6322 - categorical_accuracy: 0.2715Source path =  /home/datasets/Project_data/val ; batch size = 52

Epoch 00001: val_loss improved from inf to 1.67552, saving model to model_13_2021-11-2804_32_25.433727/model-00001-1.63224-0.27149-1.67552-0.13000.h5
13/13 [==============================] - 84s 6s/step - loss: 1.6322 - categorical_accuracy: 0.2715 - val_loss: 1.6755 - val_categorical_accuracy: 0.1300
Epoch 2/50
13/13 [==============================] - ETA: 0s - loss: 1.4537 - categorical_accuracy: 0.3696
Epoch 00002: val_loss did not improve from 1.67552
13/13 [==============================] - 73s 6s/step - loss: 1.4537 - categorical_accuracy: 0.3696 - val_loss: 1.6916 - val_categorical_accuracy: 0.2200
Epoch 3/50
13/13 [==============================] - ETA: 0s - loss: 1.3442 - categorical_accuracy: 0.4411
Epoch 00003: val_loss did not improve from 